In [ ]:
# mistakes in aff wit disclosure, article have its own category, missing affiliation , auth mismatch, abs text missing

In [ ]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import random

dic = {
    'source_id':[],
    'manual_id':[],
    'article_title':[],
    'url':[],
    'authors':[],
    'author_affiliation':[],
    'abstract_text':[],
    'date':[],
    'start_time':[],
    'end_time':[],
    'location':[],
    'session_id':[],
    'news_type':[],
    'session_title':[],
    'session_type':[],
    'category':[],
    'sub_category':[],
    'disclosure':[]
    }

req1 = requests.get("https://www.eventscribe.net/2023/SIR23/agenda.asp?pfp=BrowsebyDay")
soup = BeautifulSoup(req1.content, 'html.parser')

In [ ]:
se = set()
c = 0
for i in soup.find_all(class_=re.compile('list-group-item.*')):
    
    
    session_number = f'S{c+1}'
    code = i.get('data-presid')
    print(code)
    if code == '1177784':
        
        if str(code) == 'None':
            continue
        icon = ''
        if i.find_all('img'): # check if there is any icon
            print('yesss iconn......')
            for i in i.find_all('img'):
                icon += i.get('title') + '; '

        session_url = f'https://www.eventscribe.net/2023/SIR23/fsPopup.asp?Mode=sessioninfo&PresentationID={code}'
        if code in se:
            print('in se :-', code)
            continue
        req = requests.get(session_url)
        soup = BeautifulSoup(req.content, 'html.parser')


        print(session_url)
        session_title = soup.find('h1').text.strip()
        session_date = soup.find(class_='pull-left pres-tidbit tipsytip').text.strip()
        ses_time_loc = soup.find_all(class_='pull-left pres-tidbit')
        session_time = ses_time_loc[0].text.strip()

        if len(ses_time_loc) > 1:
            session_loc = ses_time_loc[1].text.strip()
        else:
            print('no location....')
            session_loc = ''

        session_loc = re.sub('Location:', '', session_loc)

        sess_text = ''
        session_author = ''
        affiliation = ''
        sess_type = ''
        category = ''
        sess_disclose = ''

        cat = soup.find_all(class_='trackname innertracks')
        if len(cat) < 1:
            cat = soup.find_all(style='color:#FFFFFF;font-size:14px; margin-top:10px; margin-right:10px; background:#489CD8')

        try:
            sess_type = soup.find(style='color:#FFFFFF; font-size:14px; margin-top:10px; margin-right:10px; background:#489CD8;').text.strip()
        except:
            try:
                sess_type = soup.find(style='margin-top:10px;').text.strip()
            except:
                sess_type = ''

        for i in cat:
            category += i.text.strip() + '; '

        auth = soup.find(class_='speakers-wrap')

        for s_i in auth.find_all(class_='row'): # author affiliation
            aut = s_i.find(class_='col-xs-12 col-md-6').a.text
            aff = s_i.find(class_="text-muted prof-text")
            aff = re.split('<br.*?>', str(aff))
            aff = ' '.join(aff)
            if re.search('<p class="text-muted prof-text">(.*?)</p>', aff, flags=re.S):
                aff = re.search('<p class="text-muted prof-text">(.*?)</p>', aff, flags=re.S).group(1)
            else:
                aff = ' '
            session_author += aut.strip() + '; '
            affiliation += aff + '; '
        if soup.find_all(class_='presentation-disclosure-block'):
            for i in soup.find_all(class_='presentation-disclosure-block')[1:]: # disclosure filling

                dis = i.text.strip()
                dis = dis.replace(';', ' ')
                sess_disclose += dis + '; '

        try: # abs_text with giving spaces
            sess_text = soup.find(class_='PresentationAbstractText mar-top')
            sess_text = str(sess_text)
            sess_text = BeautifulSoup(sess_text, 'html.parser').text
        except:
            sess_text = ''
        if str(sess_text) == 'None':
            sess_text = ''

        if session_title == sess_type:
            sess_type = ' '
            print('check if.......')
            
        if re.search('Disclosure|disclosu', affiliation, flags=re.S):
            affiliation = re.sub('Disclosure.*|disclosu.*', '', affiliation, flags=re.S)

        category = re.sub(sess_type+'; ', '', category)
        print(f'session title :- {session_title}')
        print(f'session type :- {sess_type}')
        print(f'category :- {category}')
        print(f'session date :- {session_date}')
        print(f'session time :- {session_time}')
        print(f'session location :- {session_loc}')
        print(f'session author :- {session_author}')
        print(f'session aff :- {affiliation}')
        print(f'session text :- {sess_text}')

        print('======================='*5)

        dic['session_id'].append(session_number)
        dic['news_type'].append('Session')
        dic['source_id'].append('')
        dic['manual_id'].append('')
        dic['article_title'].append(session_title)
        dic['url'].append(session_url)
        dic['authors'].append(session_author)
        dic['author_affiliation'].append(affiliation)
        dic['abstract_text'].append(sess_text)
        dic['date'].append(session_date)
        dic['start_time'].append(session_time.split('–')[0])
        dic['end_time'].append(session_time.split('–')[1])
        dic['location'].append(session_loc)
        dic['session_title'].append(session_title)
        dic['session_type'].append(sess_type)
        dic['category'].append(category)
        dic['sub_category'].append('')
        dic['disclosure'].append(sess_disclose)


        for i in re.split("li class='list-group-item '", req.text)[1:]: # article links

            url = re.search("data-presid='(.*?)' data-url", i).group(1)
            article_url = f'https://www.eventscribe.net/2023/SIR23/fsPopup.asp?Mode=presInfo&PresentationID={url}'
            print(article_url)
            req_art = requests.get(article_url)
            soup = BeautifulSoup(req_art.content, 'html.parser')

            se.add(url)
            article_cat = category
            cat = soup.find_all(style='background:#489CD8; color:#FFFFFF; font-size:14px; margin-top:10px; margin-right:10px;')
            if len(cat) < 1:
                cat = soup.find_all(style='color:#FFFFFF;font-size:14px; margin-top:10px; margin-right:10px; background:#489CD8')

            for i in cat:
                article_cat += i.text.strip() + '; '

            article_title = soup.find('h1', style='margin-top:10px;').text.strip()
            article_date = soup.find('div', class_='pull-left pres-tidbit tipsytip').text.strip()
            article_author = ''
            article_aff = ''
            article_abs_text = ''
            art_disclose = ''

            time_loc = soup.find_all(class_='pull-left pres-tidbit')
            art_time = time_loc[0].text.strip()
            art_auth = soup.find(class_='speakers-wrap')

            for a_i in art_auth.find_all(class_='row'): # author affiliation
                aut = a_i.find(class_='col-xs-12 col-md-6').a.text
                aff = a_i.find(class_="text-muted prof-text")
                aff = re.split('<br.*?>', str(aff))
                aff = ' '.join(aff)
                if re.search('<p class="text-muted prof-text">(.*?)</p>', aff, flags=re.S):
                    aff = re.search('<p class="text-muted prof-text">(.*?)</p>', aff, flags=re.S).group(1)
                else:
                    aff = ' '
                article_author += aut.strip() + '; '
                article_aff += aff + '; '

            if soup.find_all(class_='presentation-disclosure-block'):
                for i in soup.find_all(class_='presentation-disclosure-block')[1:]: # disclosure filling

                    dis = i.text.strip()
                    dis = dis.replace(';', ' ')
                    art_disclose += dis + '; '
            try:
                article_abs_text = soup.find(class_='PresentationAbstractText mar-top')
                article_abs_text = str(article_abs_text)
                article_abs_text = article_abs_text.replace('<b>', '')
                article_abs_text = article_abs_text.replace('</b>', '')
                article_abs_text = re.split('<br.*?>', article_abs_text)
                article_abs_text = ' '.join(article_abs_text)
                article_abs_text = re.search('<div class="PresentationAbstractText mar-top">(.*?)</div>', article_abs_text).group(1)
                article_abs_text = BeautifulSoup(article_abs_text, 'html.parser').text
            except:
                article_abs_text = ''
            
            if re.search('Disclosure|disclosu', article_aff, flags=re.S):
                article_aff = re.sub('Disclosure.*|disclosu.*', '', article_aff, flags=re.S)

            print(article_title)
            print(article_date)
            print(art_time)
            print(article_author)
            print(article_aff)
            print(f'article text :- {article_abs_text}')
            print(f'dusra column :- {article_cat}')

            dic['session_id'].append(session_number)
            dic['news_type'].append('Abstract')
            dic['source_id'].append('')
            dic['manual_id'].append('')
            dic['article_title'].append(article_title)
            dic['url'].append(article_url)
            dic['authors'].append(article_author)
            dic['author_affiliation'].append(article_aff)
            dic['abstract_text'].append(article_abs_text)
            dic['date'].append(article_date)
            dic['start_time'].append(art_time.split('–')[0])
            try:
                dic['end_time'].append(art_time.split('–')[1])
            except:
                dic['end_time'].append('sess ka' + session_time.split('–')[1])
            dic['location'].append(session_loc)
            dic['session_title'].append(session_title)
            dic['session_type'].append(sess_type)
            dic['category'].append(category)
            dic['sub_category'].append('')
            dic['disclosure'].append(art_disclose)
       
    c += 1
    print(c)

In [ ]:
df = pd.DataFrame(dic)
df

In [ ]:
df.to_excel('SIR_all.xlsx', index=False)